## Factor Graph and iSAM2 Numeric Example 
Below is a **complete "small-but-real" iSAM2 example** that actually shows the advantage over batch pose-graph optimization.
I'll use **10 poses** (enough that "incremental + partial refactor" is visible), with:

* Poses $X_0,\dots,X_9$ in SE(2)
* Odometry factors for every consecutive pair
* One loop closure that connects **two nearby late poses** (not back to $X_0$), so only a **suffix** of the trajectory needs to be updated

We will visit:

✅ the **full factor-graph equations**
✅ the **linearization and square-root/QR equations**
✅ what batch pose-graph does vs what iSAM2 does
✅ exactly **which part** is different and better in iSAM2

---

## 0) State, operators, and measurement model

Each pose:

$$
X_i = (x_i, y_i, \theta_i), \quad p_i=\begin{bmatrix}x_i \\ y_i\end{bmatrix}
$$

Relative-pose prediction:

$$
h(X_i,X_j)=
\begin{bmatrix}
R(\theta_i)^T(p_j-p_i) \\
\mathrm{wrap}(\theta_j-\theta_i)
\end{bmatrix}
$$

Residual for a measurement $z_{ij}=(t_{ij},\Delta\theta_{ij})$:

$$
r_{ij}(X_i,X_j)=
\begin{bmatrix}
R(\theta_i)^T(p_j-p_i)-t_{ij} \\
\mathrm{wrap}((\theta_j-\theta_i)-\Delta\theta_{ij})
\end{bmatrix}
\in\mathbb{R}^3
$$

---

## 1) The data: 10 poses, odometry, one loop closure

### 1.1 Anchor (prior) on $X_0$

$$
z_0=(0,0,0)
$$

$$
r_0(X_0)=
\begin{bmatrix}
x_0 \\ y_0 \\ \theta_0
\end{bmatrix}
$$

### 1.2 Odometry factors ($z_{i,i+1}$) for $i=0..8$

We drive forward ~1 m each step, with tiny heading drift:

$$
z_{i,i+1}=(1.00,\ 0.00,\ 0.01)\ \ \text{for } i=0,\dots,8
$$

### 1.3 Loop closure factor (this is the "interesting" part)

Connect pose 9 to pose 6 (a **local** closure late in the trajectory):

Measured relative pose of 9 expressed in frame 6:

$$
z_{6,9}=(3.00,\ 0.10,\ 0.00)
$$

Interpretation: from $X_6$ you should see $X_9$ about 3 m ahead, slight lateral offset, almost no relative yaw.

---

## 2) Noise / information matrices (same style as before)

Odometry (moderate):

* $\sigma_{x,y}^{\text{odom}}=0.05$ m
* $\sigma_{\theta}^{\text{odom}}=0.05$ rad

$$
\Omega_{\text{odom}}=\mathrm{diag}(400,\ 400,\ 400)
$$

Loop closure (tighter):

* $\sigma_{x,y}^{\text{loop}}=0.02$ m
* $\sigma_{\theta}^{\text{loop}}=0.02$ rad

$$
\Omega_{\text{loop}}=\mathrm{diag}(2500,\ 2500,\ 2500)
$$

Strong prior:

$$
\Omega_0=\mathrm{diag}(10^6,\ 10^6,\ 10^6)
$$

---

## 3) Build initial guess by integrating odometry

Set:

$$
X_0=(0,0,0)
$$

For $i=0..8$:

$$
p_{i+1}=p_i + R(\theta_i)\begin{bmatrix}1 \\ 0\end{bmatrix}
,\quad
\theta_{i+1}=\theta_i+0.01
$$

This yields approximately a gentle arc. For the loop-closure demonstration, we only need approximate values at $X_6$ and $X_9$.

Compute headings:

$$
\theta_6=0.06,\quad \theta_9=0.09
$$

Approx positions (using $\sum \cos\theta_k$, $\sum \sin\theta_k$):

$$
x_6 \approx 5.9925,\quad y_6 \approx 0.1499
$$
$$
x_9 \approx 8.9636,\quad y_9 \approx 0.3585
$$

So:

$$
X_6 \approx (5.9925,\ 0.1499,\ 0.06)
,\quad
X_9 \approx (8.9636,\ 0.3585,\ 0.09)
$$

---

## 4) Factor graph objective (full equation)

Variables:

$$
\mathcal X=\{X_0,\dots,X_9\}
$$

Factors:

* prior ($f_0$)
* odometry factors ($f_{0,1},\dots,f_{8,9}$)
* loop closure factor ($f_{6,9}$)

Objective:

$$
\min_{\mathcal X}\
r_0(X_0)^T\Omega_0 r_0(X_0)
+
\sum_{i=0}^{8} r_{i,i+1}(X_i,X_{i+1})^T\Omega_{\text{odom}} r_{i,i+1}(X_i,X_{i+1})
+
r_{6,9}(X_6,X_9)^T\Omega_{\text{loop}} r_{6,9}(X_6,X_9)
$$

This is the same "pose graph" objective; factor-graph notation just makes the solver structure clearer.

---

## 5) Linearization and Jacobians (what both batch GN and iSAM2 use)

At current estimate $\mathcal X^k$, each factor linearizes:

$$
r_f(\mathcal X^k \oplus \delta x)\approx r_f^k + J_f \delta x_f
$$

For a binary factor $(i,j)$:

$$
r_{ij}\approx r_{ij}^k + A_{ij}\delta X_i + B_{ij}\delta X_j
$$

Jacobians (same as you used before):

$$
B_{ij}=
\begin{bmatrix}
R(\theta_i)^T & 0 \\
0\ 0 & 1
\end{bmatrix}
$$

$$
A_{ij}=
\begin{bmatrix}
R(\theta_i)^T & d_{ij} \\
0\ 0 & -1
\end{bmatrix}
$$

where

$$
d_{ij}=
\left(\frac{\partial R(\theta_i)^T}{\partial \theta_i}\right)(p_j-p_i)
$$

$$
\frac{\partial R(\theta)^T}{\partial \theta}=
\begin{bmatrix}
-\sin\theta & \cos\theta \\
-\cos\theta & -\sin\theta
\end{bmatrix}
$$

---

## 6) Compute the loop-closure residual numerically (so it's concrete)

Prediction:

$$
\hat t_{6,9}=R(\theta_6)^T(p_9-p_6),\quad \hat\theta_{6,9}=\mathrm{wrap}(\theta_9-\theta_6)
$$

Compute:

$$
p_9-p_6=
\begin{bmatrix}
8.9636-5.9925 \\
0.3585-0.1499
\end{bmatrix}
= \begin{bmatrix}
2.9711 \\
0.2086
\end{bmatrix}
$$

With $\theta_6=0.06$, $c=\cos0.06\approx0.9982$, $s=\sin0.06\approx0.0600$,

$$
R(\theta_6)^T=
\begin{bmatrix}
c & s \\
-s & c
\end{bmatrix}
$$

$$
\hat t_{6,9}\approx
\begin{bmatrix}
0.9982 & 0.0600 \\
-0.0600 & 0.9982
\end{bmatrix}
\begin{bmatrix}
2.9711 \\
0.2086
\end{bmatrix}
\approx
\begin{bmatrix}
2.978 \\
0.030
\end{bmatrix}
$$

$$
\hat\theta_{6,9}=\theta_9-\theta_6=0.03
$$

Measured:

$$
z_{6,9}=
\begin{bmatrix}
3.00 \\
0.10 \\
0.00
\end{bmatrix}
$$

Residual (prediction minus measurement):

$$
r_{6,9}^k=
\begin{bmatrix}
2.978-3.00 \\
0.030-0.10 \\
0.03-0.00
\end{bmatrix}
= \begin{bmatrix}
-0.022 \\
-0.070 \\
0.030
\end{bmatrix}
$$

This is the "new information" that will pull the estimate.

---

## 7) Batch pose-graph solve (what changes when loop closure arrives)

Batch GN forms the global system over all increments:

$$
\delta x=
\begin{bmatrix}
\delta X_0 \\ \delta X_1 \\ \vdots \\ \delta X_9
\end{bmatrix}\in\mathbb{R}^{30}
$$

Stack all factors into one big weighted least squares:

$$
\min_{\delta x}\ |W(J\delta x + r)|^2
$$

Normal equations:

$$
H\delta x=-b
$$
$$
H=J^T\Omega J,\quad b=J^T\Omega r
$$

✅ Works
❌ When loop closure arrives, you typically **relinearize**, **rebuild** $(H,b)$, and **solve the full 30×30** system again (or at least refactor the whole sparse system).

---

## 8) iSAM2 solve (the same math, different representation + incremental update)

iSAM2 uses the **square-root form** and maintains an **incremental QR factorization**.

### 8.1 Square-root form

Factor the information:

$$
\Omega = W^T W
$$

Define:

$$
\tilde J = WJ,\quad \tilde r = Wr
$$

Solve:

$$
\min_{\delta x}\ |\tilde J \delta x + \tilde r|^2
$$

### 8.2 QR factorization (what iSAM2 maintains incrementally)

$$
\tilde J = QR
$$

Then:

$$
\min_{\delta x}\ |R\delta x + Q^T\tilde r|^2
$$

The "drop orthonormal part" step is:

$$
|Qy|^2 = y^TQ^TQy = |y|^2
$$

So the solve is:

$$
R\delta x = -Q^T\tilde r
$$

### 8.3 The real iSAM2 trick: Bayes tree + partial refactor

iSAM2 interprets the triangular system $R$ as a **Bayes net**, and groups it into a **Bayes tree**.

When a new factor arrives (here $f_{6,9}$), iSAM2:

1. Inserts the new factor (adds a few rows to $\tilde J$)
2. Finds which variables are involved: $\{X_6,X_9\}$
3. Computes the **affected cliques** in the Bayes tree (typically a subset)
4. **Re-eliminates only that subset**, reusing the rest
5. Optionally relinearizes only variables whose $|\delta X_i|$ exceeds a threshold

---

## 9) Why this example shows iSAM2's advantage

Because the loop closure is **local** (6↔9), it does *not* necessarily force refactorization of the entire trajectory, depending on ordering / tree structure.

A typical elimination ordering for a chain is close to:

$$
X_0 \rightarrow X_1 \rightarrow \cdots \rightarrow X_9
$$

Under such an ordering, the Bayes tree often looks like a chain of cliques. Adding a constraint between $X_6$ and $X_9$ tends to affect mostly the **suffix** $\{X_6,X_7,X_8,X_9\}$, not $\{X_0..X_5\}$.

So:

* Batch: refactor the whole 30×30 (or large portion)
* iSAM2: refactor only the "touched" Bayes-tree region

On a 10-pose toy problem the speedup is modest, but the *mechanism* is exactly what makes iSAM2 scale to tens of thousands of poses.

---

## 10) Exactly which parts of iSAM2 are different and better

### ✅ Difference 1: Solve form (square-root QR instead of normal equations)

Batch often goes through:

$$
H=J^T\Omega J
$$

iSAM2 stays in:

$$
\tilde J = WJ,\quad \tilde J=QR
$$

**Better because**: QR is numerically stable and avoids explicitly forming $H$, which can worsen conditioning.

---

### ✅ Difference 2: Incremental update of the factorization

Batch: after each new factor, refactor a large system.
iSAM2: update $R$ incrementally by modifying only affected parts.

Mathematically: iSAM2 maintains an evolving factorization of the same least-squares system rather than recomputing it.

---

### ✅ Difference 3: Bayes tree (localized re-elimination)

New factor touches only certain variables, so only those cliques are rebuilt.

This is the big win in SLAM with long trajectories + occasional loop closures.

---

### ✅ Difference 4: Partial relinearization

iSAM2 does not relinearize everything every time; it uses a threshold:

$$
|\delta X_i| > \epsilon \ \Rightarrow\ \text{relinearize } X_i
$$

This saves time while keeping accuracy.

---

## 11) What I didn't do (on purpose) and what I can do next

I didn't expand the full 30×30 numeric $H$ and solve it again because it becomes a wall of numbers and doesn't clarify iSAM2's unique benefit.

If you want the most "hands-on" next step, I can do either of these:

1. **Show the exact affected set** in a specific Bayes tree ordering (with cliques) for this 10-pose graph, and illustrate which cliques get re-eliminated when adding $f_{6,9}$.
2. Build a "streaming" timeline: add odometry one-by-one, run iSAM2 update equations each time, then add loop closure and show what is reused vs recomputed.
3. If you want the full numeric solve anyway, I can provide the explicit **stacked $\tilde J$** structure (block-sparse) and show how QR elimination proceeds variable-by-variable.

Pick 1), 2), or 3) and I'll continue with that path.
```

**Key changes made:**
1. Replaced all `(` with `$` for inline math
2. Replaced all `[` with `$$` for display math
3. Fixed set notation using `\{` and `\}`
4. Properly escaped all LaTeX commands
5. Added proper line breaks in matrices and equations
6. Fixed matrix formatting and alignment
7. Used `\begin{bmatrix}` consistently for matrices
8. Properly formatted all mathematical expressions

This version should render correctly in Jupyter Notebook.

## 1) Factor graph objects for this example

### Variables (nodes)

We keep the same SE(2) poses:

$$
X_i = (x_i, y_i, \theta_i)\in \mathbb{R}^2\times S^1,\quad i\in\{0,1,2,3\}
$$

Collect all variables:

$$
\mathcal{X}=\{X_0,X_1,X_2,X_3\}
$$

Gauge fixing (same as before): anchor $X_0$ by a **prior factor** (more on this below).

---

### Factors (constraints)

Each measurement creates a factor connected to the variables it depends on.

We have 4 measurements:

* Odometry (relative pose) factors: $f_{01}, f_{12}, f_{23}$
* Loop closure (relative pose) factor: $f_{30}$
* Plus a **prior** factor ($f_0$) to fix gauge

So the factor set is:

$$
\mathcal{F}=\{f_0,\ f_{01},\ f_{12},\ f_{23},\ f_{30}\}
$$

---

## 2) Measurement model (the factor "prediction")

For any edge $(i,j)$, the predicted relative pose is:

$$
h(X_i,X_j)=
\begin{bmatrix}
R(\theta_i)^T(p_j-p_i) \\
\mathrm{wrap}(\theta_j-\theta_i)
\end{bmatrix}
\in \mathbb{R}^3
$$

where $p_i=[x_i\ y_i]^T$.

The residual used by that factor is:

$$
r_{ij}(X_i,X_j)=
\begin{bmatrix}
R(\theta_i)^T(p_j-p_i)-t_{ij} \\
\mathrm{wrap}\left((\theta_j-\theta_i)-\Delta\theta_{ij}\right)
\end{bmatrix}
$$

and the measurement is $z_{ij}=(t_{ij},\Delta\theta_{ij})$.

For your numbers:

* $z_{01}=(1.10,0.00,0.05)$
* $z_{12}=(1.00,0.00,0.02)$
* $z_{23}=(0.00,1.10,1.70)$
* $z_{30}=(-1.00,2.00,-\pi/2)$

---

## 3) Noise model: covariance and information per factor

Each factor has covariance $\Sigma_{ij}$ and information $\Omega_{ij}=\Sigma_{ij}^{-1}$.

Use the same values as before:

$$
\Omega_{\text{odom}}=\mathrm{diag}(400,\ 400,\ 131.312254)
$$

$$
\Omega_{\text{loop}}=\mathrm{diag}(2500,\ 2500,\ 820.7015875)
$$

And the prior factor on $X_0$ (very strong to anchor):

Let the prior measurement be:

$$
z_0=(0,0,0)
$$

Residual:

$$
r_0(X_0)=
\begin{bmatrix}
x_0 \\ y_0 \\ \theta_0
\end{bmatrix}
$$

Choose (example) a very strong prior:

$$
\Omega_0=\mathrm{diag}(10^6,\ 10^6,\ 10^6)
$$

(Or you can hard-fix $X_0$; the math becomes equivalent.)

---

## 4) Factor graph objective function

A factor graph is defined by the **sum of squared weighted residuals**:

$$
\min_{\mathcal{X}} \ \sum_{f\in\mathcal{F}} |r_f(\mathcal{X}_f)|_{\Omega_f}^2
$$

with

$$
|r|_{\Omega}^2 = r^T\Omega r
$$

For this specific graph:

$$
\min_{X_0,X_1,X_2,X_3}
\left(
r_0(X_0)^T\Omega_0 r_0(X_0)
+
\sum_{(i,j)\in\{(0,1),(1,2),(2,3)\}} r_{ij}(X_i,X_j)^T\Omega_{\text{odom}} r_{ij}(X_i,X_j)
+
r_{30}(X_3,X_0)^T\Omega_{\text{loop}} r_{30}(X_3,X_0)
\right)
$$

That's the factor-graph equation.

---

## 5) The graph structure (how it "looks")

* Variable nodes: circles $(X_0,X_1,X_2,X_3)$
* Factor nodes: squares $(f_0, f_{01}, f_{12}, f_{23}, f_{30})$

Connections:

* $f_0 \leftrightarrow X_0$
* $f_{01}\leftrightarrow (X_0,X_1)$
* $f_{12}\leftrightarrow (X_1,X_2)$
* $f_{23}\leftrightarrow (X_2,X_3)$
* $f_{30}\leftrightarrow (X_3,X_0)$

This is the exact same information you had in the pose graph; factor-graph notation just makes the "prior" and future extensions (landmarks, IMU biases, time offsets, etc.) natural.

---

## 6) Linearization (one Gauss-Newton iteration) in factor-graph form

Pick a current estimate $\mathcal{X}^k$ (your odometry-integrated initial guess).

Linearize each factor residual around $\mathcal{X}^k$:

### Unary factor (prior)

$$
r_0(X_0^k\oplus \delta X_0)\approx r_0(X_0^k)+J_0\delta X_0
$$

with

$$
J_0=\frac{\partial r_0}{\partial X_0} = I_{3\times 3}
$$

### Binary factor (relative pose)

$$
r_{ij}(X_i^k\oplus \delta X_i,\ X_j^k\oplus \delta X_j)
\approx
r_{ij}^k + A_{ij}\delta X_i + B_{ij}\delta X_j
$$

where

$$
A_{ij}=\frac{\partial r_{ij}}{\partial X_i},\quad
B_{ij}=\frac{\partial r_{ij}}{\partial X_j}
$$

These are exactly the Jacobians you used before (same formulas; same numeric values for the loop-closure Jacobian at node 3).

---

## 7) Building the global system from factors (this is "we are building it")

Stack all increments:

$$
\delta x =
\begin{bmatrix}
\delta X_0 \\ \delta X_1 \\ \delta X_2 \\ \delta X_3
\end{bmatrix}\in\mathbb{R}^{12}
$$

Each factor contributes small blocks into a global normal equation:

$$
H\delta x = -g
$$

Where for each factor $f$ with linearization $r_f \approx r_f^k + J_f \delta x_f$:

$$
H \mathrel{+}= J_f^T\Omega_f J_f
$$
$$
g \mathrel{+}= J_f^T\Omega_f r_f^k
$$

* For the prior: $J_f$ touches only $X_0$ (a 3×3 block).
* For an edge $(ij)$: $J_f=[A_{ij}\ \ B_{ij}]$ touching blocks for $X_i$ and $X_j$.

This is the factor-graph "assembly rule".

If you hard-fix $X_0$, you drop $\delta X_0$ and you get the same 9×9 system you built earlier.

---

## 8) What is the practical difference vs your pose-graph writeup?

✅ Same math for this example (because it's a pure pose graph).

The differences are conceptual and become huge once you add more variable types:

1. **Pose graph** viewpoint

* Variables: only poses
* Factors: only relative pose constraints
* Gauge: often "fix first pose" by removing it from the state

2. **Factor graph** viewpoint

* Variables: anything (poses, landmarks, IMU biases, velocities, time offset, camera intrinsics, etc.)
* Factors: any measurement model (unary, binary, ternary, high-arity)
* Gauge: naturally represented as a **prior factor** (soft) or hard constraint

So: pose graph is a special case of factor graph.

---

## 9) Minimal "build it" pseudocode (structure only)

* Add variables: $(X_0,X_1,X_2,X_3)$
* Add factors:
  * prior factor on $X_0$
  * between factors for odometry edges $(0,1), (1,2), (2,3)$
  * between factor for loop closure $(3,0)$
* Run GN/LM:
  * linearize each factor → $(r_f^k, J_f)$
  * accumulate $(H,g)$
  * solve $H\delta x=-g$
  * update variables

---

If you want, I can take **your exact numeric values** and write the factor-graph system as an explicit **stacked Jacobian matrix $(J)$** and residual vector $(r)$ so you literally see:

$$
\min_{\delta x} |J\delta x + r|_{\Omega}^2
$$

and how it becomes $H=J^T\Omega J$, $g=J^T\Omega r$.



## Factor graph vs pose graph
A factor graph can be seen as a generalization of a pose graph when considered as a representation for graph based slam. In a factor graph, we can model more than with pose graphs a such as factors connected to a single node, for example useful for gps/gas’s measurements.


- Factor Graph - 5 Minutes with Cyrill
Refs: [1](https://www.youtube.com/watch?v=uuiaqGLFYa4&t=145s)


Odometery Measurment:
IMU
Wheel 
Visual Odometery


## Factor Graph - 5 Minutes with Cyrill
Refs: [1](https://www.youtube.com/watch?v=uuiaqGLFYa4&t=145s)

## Georgia Tech Smoothing and Mapping Library  
Refs [1](https://gtsam.org/), [2](https://github.com/borglab/gtsam)


## iSAM: Incremental Smoothing and Mapping  
Refs: [1](https://openslam-org.github.io/iSAM)


## python-graphslam

Refs: [1](https://github.com/JeffLIrion/python-graphslam), [2](https://python-graphslam.readthedocs.io/en/stable/), [3](https://www.youtube.com/watch?v=zOr9HreMthY), [4](https://github.com/gisbi-kim/modern-slam-tutorial-python/b)


## GTSAM python Tutorial Robust Pose-graph Optimization 

Refs: [1](https://github.com/gisbi-kim/modern-slam-tutorial-python), [2](https://www.youtube.com/watch?v=zOr9HreMthY)


Graph Pose SLAM and Factor Graph SLAM are closely related, as both are used for solving SLAM (Simultaneous Localization and Mapping) problems using graph-based optimization. However, they differ in their conceptual modeling, mathematical formulation, and representation of the problem. Here's a detailed comparison:

---

### 1. **Graph Pose SLAM**
Graph Pose SLAM is a traditional graph-based approach where:
- **Nodes (Vertices):** Represent robot poses ($ \textbf{X}_i = (x_i, y_i, \theta_i) $) in the trajectory or landmarks in the environment.
- **Edges:** Represent spatial constraints between nodes, which are derived from odometry or sensor measurements.

#### Key Features:
1. **Structure:**
   - A pose graph connects robot poses with odometry constraints (between consecutive poses) and loop-closure constraints (from sensor observations).
   - The optimization minimizes the error between the predicted and measured transformations (relative pose constraints).

2. **Error Formulation:**
   - Each edge $e_{ij}$ corresponds to the error between:
     - $z_{ij}$: The observed relative transformation between $ \textbf{X}_i $ and $ \textbf{X}_j $.
     - $h(\textbf{X}_i, \textbf{X}_j)$: The predicted relative transformation based on current pose estimates.
   - The optimization minimizes the total error:
     $
     E = \frac{1}{2} \sum_{ij} e_{ij}^T \Omega_{ij} e_{ij}
     $
   - Here, $ \Omega_{ij} $ is the information matrix (inverse of the measurement covariance).

3. **Application:**
   - Primarily used for solving pose graph SLAM problems, where the robot's trajectory is optimized given odometry and sensor constraints.

4. **Example:**
   - In Graph Pose SLAM, the problem is solved iteratively using approaches like Gauss-Newton or Levenberg-Marquardt, where $H$ and $b$ (information matrix and residual vector) are constructed from the graph.

---

### 2. **Factor Graph SLAM**
Factor Graph SLAM is a more general framework derived from probabilistic graphical models. It represents the SLAM problem as a **factor graph**:
- **Variables (Nodes):** Represent unknowns to be estimated, including robot poses ($ \textbf{X}_i $) and possibly landmarks ($ \textbf{L}_j $).
- **Factors:** Represent probabilistic constraints or measurements relating variables (e.g., odometry, sensor data).

#### Key Features:
1. **Structure:**
   - A factor graph explicitly separates variables and factors:
     - Variables ($ \textbf{X}_i, \textbf{L}_j $): Robot poses and landmarks.
     - Factors ($ f $): Constraints or measurements, such as odometry ($f_\text{odom}$) or observations ($f_\text{obs}$).
   - Each factor connects one or more variables.

2. **Error Formulation:**
   - Factor graph SLAM minimizes the negative log of the joint probability of all variables given the measurements:
     $
     \mathcal{P}(\mathbb{X}, \mathbb{L} \mid \mathbb{Z}) \propto \prod_k f_k(\mathbb{X}, \mathbb{L}, \mathbb{Z})
     $
   - The negative log transforms this into a sum of squared errors:
     $
     \mathcal{E} = \sum_k \| e_k(\mathbb{X}, \mathbb{L}) \|^2
     $
   - Each factor contributes an error term $e_k$, depending on the associated measurement and variables.

3. **Generalization:**
   - Factor graphs are more general and can handle a variety of problems, including SLAM with landmarks, multi-robot SLAM, and sensor fusion.

4. **Example:**
   - A factor connecting $ \textbf{X}_i $ and $ \textbf{X}_j $ via odometry $z_{ij}$:
     $
     f_\text{odom}(\textbf{X}_i, \textbf{X}_j) = \exp\left(-\| z_{ij} - h(\textbf{X}_i, \textbf{X}_j) \|^2_{\Omega_{ij}} \right)
     $

---

### **Comparison Table**

| Feature                  | Graph Pose SLAM                            | Factor Graph SLAM                      |
|--------------------------|--------------------------------------------|---------------------------------------|
| **Modeling**             | Focuses on optimizing pose graphs.         | Probabilistic framework with variables and factors. |
| **Nodes**                | Represent robot poses (and landmarks).     | Represent unknown variables (poses, landmarks). |
| **Edges**                | Represent spatial constraints.             | Represent probabilistic factors.      |
| **Error Function**       | Sum of squared errors between predictions and measurements. | Negative log-likelihood of the joint probability. |
| **Scope**                | Primarily for pose graph SLAM.             | General-purpose, supports pose SLAM, landmark SLAM, multi-robot SLAM, etc. |
| **Mathematical Framework**| Directly minimizes errors in spatial constraints. | Minimizes negative log-probability, derived from Bayesian inference. |
| **Implementation**       | Often represented as a sparse graph of poses. | Factor graph explicitly separates variables and factors. |
| **Optimization**         | Solved using sparse least-squares (e.g., Gauss-Newton). | Solved using tools like GTSAM or Ceres for factor graphs. |
| **Generality**           | Specific to poses or trajectories.         | Handles broader problems, including landmarks and hybrid graphs. |

---

### **Summary**
- **Graph Pose SLAM**: Simpler and focuses on optimizing a robot's trajectory based on odometry and sensor constraints. It directly operates on pose graphs.
- **Factor Graph SLAM**: More general and probabilistic, explicitly separating variables and measurements. It can model additional factors (e.g., landmarks, sensor biases, multi-robot interactions) and is better suited for complex SLAM problems.

Factor Graph SLAM subsumes Graph Pose SLAM as a special case.